In [1]:
import networkx as nx
import pandas as pd
import os
import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt

In [2]:
leadership_path = r'C:\Projects\connecteddatahub\data\cleaned_dataframes\master_leadershipdata.csv'
data_path = r'C:\Projects\connecteddatahub\data'
leadership_df = pd.read_csv(leadership_path)
print(leadership_df.columns)

Index(['Year', 'Name', 'Position', 'Institution', 'SubInstitution',
       'Education', 'FixedPosition', 'Seniority', 'Designation',
       'AffiliationId', 'carnegie_id', 'SystemId', 'RawName', 'RawInstitution',
       'RawPosition', 'FirstName', 'MiddleName', 'LastName', 'NickName',
       'SuffixName', 'PrefixName', 'MiddleInitials', 'RawName4Match',
       'LeaderId'],
      dtype='object')


In [3]:
board_df =  leadership_df[leadership_df['FixedPosition'] == 'Board Member']
print(board_df.shape)

(54153, 24)


In [4]:

# def build_interlock_graphs(board_df):
#     """Create node and edge DataFrames for yearly interlocks."""
#     interlocks = []

#     # person-level pairing
#     for (year, first, last), person_data in board_df.groupby(["Year", "FirstName", "LastName"]):
#         affiliations = person_data[["AffiliationId", "Institution", "SystemId"]].drop_duplicates()
#         if len(affiliations) <= 1:
#             continue

#         for (id1, inst1, sys1), (id2, inst2, sys2) in combinations(affiliations.values, 2):
#             if pd.notna(sys1) and pd.notna(sys2) and sys1 == sys2:
#                 continue
#             u, v = sorted((id1, id2))
#             interlocks.append({
#                 "Year": year,
#                 "AffiliationId_1": u,
#                 "AffiliationId_2": v,
#                 "Institution_1": inst1 if u == id1 else inst2,
#                 "Institution_2": inst2 if u == id1 else inst1,
#                 "SystemId_1": sys1 if u == id1 else sys2,
#                 "SystemId_2": sys2 if u == id1 else sys1,
#             })

#     edges = pd.DataFrame(interlocks)

#     # edge aggregation
#     edges = (
#         edges.groupby(["Year", "AffiliationId_1", "Institution_1",
#                        "AffiliationId_2", "Institution_2"], as_index=False)
#         .size()
#         .rename(columns={"size": "InterlockCount"})
#     )

#     # board sizes and normalized weight
#     board_sizes = (
#         board_df.groupby(["Year", "AffiliationId"])["Name"]
#         .nunique()
#         .reset_index(name="BoardSize")
#     )

#     edges = edges.merge(
#         board_sizes.rename(columns={"AffiliationId": "AffiliationId_1", "BoardSize": "BoardSize_1"}),
#         on=["Year", "AffiliationId_1"], how="left"
#     ).merge(
#         board_sizes.rename(columns={"AffiliationId": "AffiliationId_2", "BoardSize": "BoardSize_2"}),
#         on=["Year", "AffiliationId_2"], how="left"
#     )

#     edges["WeightNorm"] = edges["InterlockCount"] / (
#         edges["BoardSize_1"] + edges["BoardSize_2"]
#     )

#     # node list
#     nodes = (
#         board_df[["AffiliationId", "Institution", "SystemId"]]
#         .drop_duplicates()
#         .rename(columns={"AffiliationId": "Id", "Institution": "Label"})
#     )

#     return edges, nodes


# def compute_edge_weakness(G, alpha=0.7, beta=0.3):
#     """Assign 'Weakness' to each edge as 0.7*(1 - overlap) + 0.3*betweenness."""
#     # neighborhood overlap
#     overlap = {}
#     for u, v in G.edges():
#         Nu, Nv = set(G.neighbors(u)) - {v}, set(G.neighbors(v)) - {u}
#         inter = len(Nu & Nv)
#         denom = len(Nu | Nv)
#         overlap[(u, v)] = inter / (denom - 2) if denom > 2 else 0.0
#     nx.set_edge_attributes(G, overlap, "overlap")

#     # edge betweenness
#     betweenness = nx.edge_betweenness_centrality(G, normalized=True)
#     nx.set_edge_attributes(G, betweenness, "betweenness")

#     # combined weakness
#     weakness = {
#         (u, v): alpha * (1 - d.get("overlap", 0.0)) + beta * d.get("betweenness", 0.0)
#         for u, v, d in G.edges(data=True)
#     }
#     nx.set_edge_attributes(G, weakness, "weakness")
#     return weakness


# def attach_weakness(edges, nodes, alpha=0.7, beta=0.3):
#     """
#     Compute and attach edge weaknesses for each year.
#     Ensures canonical ordering (min, max) for merge alignment.
#     """
#     # enforce consistent dtypes (prevents merge misses due to int vs str)
#     edges = edges.copy()
#     edges["AffiliationId_1"] = edges["AffiliationId_1"].astype(object)
#     edges["AffiliationId_2"] = edges["AffiliationId_2"].astype(object)

#     all_weakness = []

#     for year, group in edges.groupby("Year", sort=False):
#         if group.empty:
#             continue

#         # Build undirected graph from this year's edges
#         G = nx.from_pandas_edgelist(
#             group,
#             source="AffiliationId_1",
#             target="AffiliationId_2",
#             edge_attr="WeightNorm",
#             create_using=nx.Graph(),
#         )

#         if G.number_of_edges() == 0:
#             continue

#         compute_edge_weakness(G, alpha, beta)

#         # IMPORTANT: canonicalize (min, max) before creating the weakness rows
#         rows = []
#         for u, v, d in G.edges(data=True):
#             a, b = (u, v) if u <= v else (v, u)
#             rows.append({
#                 "Year": year,
#                 "AffiliationId_1": a,
#                 "AffiliationId_2": b,
#                 "Weakness": d.get("weakness", np.nan)
#             })

#         all_weakness.append(pd.DataFrame(rows))

#     if not all_weakness:
#         # nothing computed; return edges with a Weakness column of NaN
#         edges["Weakness"] = np.nan
#         return edges

#     weakness_df = pd.concat(all_weakness, ignore_index=True)

#     # Merge by canonical IDs + Year
#     out = edges.merge(
#         weakness_df,
#         on=["Year", "AffiliationId_1", "AffiliationId_2"],
#         how="left"
#     )

#     return out


In [ ]:
def _eigvec_centrality_componentwise(G, weight="WeightNorm", max_iter=2000, tol=1e-07):
    """
    Robust eigenvector centrality for disconnected graphs:
    - Run per connected component (undirected), weighted if available.
    - Isolated nodes get 0.0.
    """
    if G.number_of_nodes() == 0:
        return {}

    scores = {n: 0.0 for n in G.nodes()}  # default for isolates / tiny comps

    for comp_nodes in nx.connected_components(G):
        H = G.subgraph(comp_nodes).copy()
        # If the component has no edges (single node), leave score=0.0
        if H.number_of_edges() == 0:
            continue
        # Try numpy (fast, stable on connected graphs)
        try:
            part = nx.eigenvector_centrality_numpy(H, weight=weight)
        except nx.AmbiguousSolution:
            # Shouldn't happen per-component, but keep belt-and-suspenders
            part = nx.eigenvector_centrality(H, weight=weight, max_iter=max_iter, tol=tol)
        except Exception:
            # Fallback to power iteration with relaxed params
            part = nx.eigenvector_centrality(H, weight=weight, max_iter=max_iter, tol=tol)
        # Write back
        for n, v in part.items():
            scores[n] = float(v)

    return scores


def _safe_betweenness(G, use_weighted_betweenness=False, weight="WeightNorm"):
    """
    Betweenness that optionally uses distance=1/weight. Works for disconnected graphs.
    """
    if not use_weighted_betweenness:
        return nx.betweenness_centrality(G, normalized=True)
    eps = 1e-12
    inv_dist = {(u, v): 1.0 / max(float(d.get(weight, 0.0)), eps) for u, v, d in G.edges(data=True)}
    nx.set_edge_attributes(G, inv_dist, name="_dist")
    bt = nx.betweenness_centrality(G, normalized=True, weight="_dist")
    # cleanup
    for u, v in G.edges():
        if "_dist" in G[u][v]:
            del G[u][v]["_dist"]
    return bt



def build_interlock_graphs(board_df):
    """Create node and edge DataFrames for yearly interlocks."""
    interlocks = []

    # person-level pairing
    for (year, first, last), person_data in board_df.groupby(["Year", "FirstName", "LastName"]):
        affiliations = person_data[["AffiliationId", "Institution", "SystemId"]].drop_duplicates()
        if len(affiliations) <= 1:
            continue

        for (id1, inst1, sys1), (id2, inst2, sys2) in combinations(affiliations.values, 2):
            # Skip if both belong to the same system
            if pd.notna(sys1) and pd.notna(sys2) and sys1 == sys2:
                continue

            # Canonicalize undirected edge
            u, v = sorted((id1, id2))
            interlocks.append({
                "Year": year,
                "AffiliationId_1": u,
                "AffiliationId_2": v,
                "Institution_1": inst1 if u == id1 else inst2,
                "Institution_2": inst2 if u == id1 else inst1,
                "SystemId_1": sys1 if u == id1 else sys2,
                "SystemId_2": sys2 if u == id1 else sys1,
            })

    edges = pd.DataFrame(interlocks)

    # edge aggregation
    edges = (
        edges.groupby(["Year", "AffiliationId_1", "Institution_1",
                       "AffiliationId_2", "Institution_2"], as_index=False)
        .size()
        .rename(columns={"size": "InterlockCount"})
    )

    # board sizes and normalized weight
    board_sizes = (
        board_df.groupby(["Year", "AffiliationId"])["Name"]
        .nunique()
        .reset_index(name="BoardSize")
    )

    edges = edges.merge(
        board_sizes.rename(columns={"AffiliationId": "AffiliationId_1", "BoardSize": "BoardSize_1"}),
        on=["Year", "AffiliationId_1"], how="left"
    ).merge(
        board_sizes.rename(columns={"AffiliationId": "AffiliationId_2", "BoardSize": "BoardSize_2"}),
        on=["Year", "AffiliationId_2"], how="left"
    )

    edges["WeightNorm"] = edges["InterlockCount"] / (
        edges["BoardSize_1"] + edges["BoardSize_2"]
    )

    # node list (year-invariant; we'll add per-year metrics later)
    nodes = (
        board_df[["AffiliationId", "Institution", "SystemId"]]
        .drop_duplicates()
        .rename(columns={"AffiliationId": "Id", "Institution": "Label"})
    )

    return edges, nodes


def compute_edge_weakness(G, alpha=0.7, beta=0.3):
    """Assign 'Weakness' to each edge as 0.7*(1 - overlap) + 0.3*betweenness."""
    # neighborhood overlap
    overlap = {}
    for u, v in G.edges():
        Nu, Nv = set(G.neighbors(u)) - {v}, set(G.neighbors(v)) - {u}
        inter = len(Nu & Nv)
        denom = len(Nu | Nv)
        overlap[(u, v)] = inter / (denom - 2) if denom > 2 else 0.0
    nx.set_edge_attributes(G, overlap, "overlap")

    # edge betweenness
    betweenness = nx.edge_betweenness_centrality(G, normalized=True)
    nx.set_edge_attributes(G, betweenness, "betweenness")

    # combined weakness
    weakness = {
        (u, v): alpha * (1 - d.get("overlap", 0.0)) + beta * d.get("betweenness", 0.0)
        for u, v, d in G.edges(data=True)
    }
    nx.set_edge_attributes(G, weakness, "weakness")
    return weakness


def attach_weakness(edges, nodes, alpha=0.7, beta=0.3):
    """
    Compute and attach edge weaknesses for each year.
    Ensures canonical ordering (min, max) for merge alignment.
    """
    edges = edges.copy()
    edges["AffiliationId_1"] = edges["AffiliationId_1"].astype(object)
    edges["AffiliationId_2"] = edges["AffiliationId_2"].astype(object)

    all_weakness = []

    for year, group in edges.groupby("Year", sort=False):
        if group.empty:
            continue

        # Build undirected graph from this year's edges
        G = nx.from_pandas_edgelist(
            group,
            source="AffiliationId_1",
            target="AffiliationId_2",
            edge_attr="WeightNorm",
            create_using=nx.Graph(),
        )

        if G.number_of_edges() == 0:
            continue

        compute_edge_weakness(G, alpha, beta)

        rows = []
        for u, v, d in G.edges(data=True):
            a, b = (u, v) if u <= v else (v, u)
            rows.append({
                "Year": year,
                "AffiliationId_1": a,
                "AffiliationId_2": b,
                "Weakness": d.get("weakness", np.nan)
            })
        all_weakness.append(pd.DataFrame(rows))

    if not all_weakness:
        edges["Weakness"] = np.nan
        return edges

    weakness_df = pd.concat(all_weakness, ignore_index=True)

    out = edges.merge(
        weakness_df,
        on=["Year", "AffiliationId_1", "AffiliationId_2"],
        how="left"
    )
    return out


# =========================================================
# 2) NEW: attach_node_metrics -> returns NODES DF (by Year)
# =========================================================

def _shannon_entropy_bits(values):
    """Base-2 Shannon entropy over nonnegative weights."""
    vals = np.asarray(list(values), dtype=float)
    s = vals.sum()
    if s <= 0 or vals.size == 0:
        return 0.0
    p = vals / s
    p = p[p > 0]
    return float(-np.sum(p * np.log2(p)))


def attach_node_metrics(edges, nodes, weight_col="WeightNorm", use_weighted_betweenness=False):
    """
    Compute node metrics per Year and return an UPDATED nodes DataFrame.
    Output is one row per (Year, Id) with node attributes + metrics.

    Metrics added:
      - degree
      - degree_centrality
      - eigenvector_centrality (weighted by `weight_col`)
      - betweenness_centrality (optionally weighted via distance = 1/weight)
      - clustering (local, unweighted)
      - modularity_class (greedy modularity)
      - interlock_entropy_bits (entropy over incident edge weights)
      - strength (sum of incident edge weights)
    """
    if edges.empty:
        base_cols = ["Year","Id","Label","SystemId","degree","degree_centrality",
                     "eigenvector_centrality","betweenness_centrality",
                     "clustering","modularity_class","interlock_entropy_bits","strength"]
        return pd.DataFrame(columns=base_cols)

    edges = edges.copy()
    edges["AffiliationId_1"] = edges["AffiliationId_1"].astype(object)
    edges["AffiliationId_2"] = edges["AffiliationId_2"].astype(object)

    out = []

    for year, gdf in edges.groupby("Year", sort=False):
        if gdf.empty:
            continue

        edge_attr = [weight_col] if (weight_col in gdf.columns) else None
        G = nx.from_pandas_edgelist(
            gdf,
            source="AffiliationId_1",
            target="AffiliationId_2",
            edge_attr=edge_attr,
            create_using=nx.Graph()
        )
        if G.number_of_nodes() == 0:
            continue

        # Degree & degree centrality
        degree = dict(G.degree())
        degree_c = nx.degree_centrality(G)

        eig_c = _eigvec_centrality_componentwise(G, weight=weight_col, max_iter=2000, tol=1e-7)
        betw = _safe_betweenness(G, use_weighted_betweenness=use_weighted_betweenness, weight=weight_col)

        # Betweenness (optionally weighted via distance = 1/weight)
        if use_weighted_betweenness and G.number_of_edges() > 0:
            eps = 1e-12
            inv_dist = {(u, v): 1.0 / max(float(d.get(weight_col, 0.0)), eps) for u, v, d in G.edges(data=True)}
            nx.set_edge_attributes(G, inv_dist, name="_dist")
            betw = nx.betweenness_centrality(G, normalized=True, weight="_dist")
            for u, v in G.edges():
                if "_dist" in G[u][v]:
                    del G[u][v]["_dist"]
        else:
            betw = nx.betweenness_centrality(G, normalized=True)

        # Local clustering (unweighted)
        clust = nx.clustering(G)

        # Greedy modularity -> integer community labels
        communities = list(nx.algorithms.community.greedy_modularity_communities(G, weight=weight_col))
        node2comm = {}
        for cid, comm in enumerate(communities):
            for n in comm:
                node2comm[n] = cid

        # Strength + entropy over incident weights
        strength = {}
        entropy_bits = {}
        for n in G.nodes():
            nbr_w = [float(G[n][nbr].get(weight_col, 1.0)) for nbr in G.neighbors(n)]
            strength[n] = float(np.sum(nbr_w)) if nbr_w else 0.0
            entropy_bits[n] = _shannon_entropy_bits(nbr_w)

        # Assemble rows (Year, Id)
        rows = []
        for n in G.nodes():
            rows.append({
                "Year": year,
                "Id": n,
                "degree": degree.get(n, 0),
                "degree_centrality": degree_c.get(n, 0.0),
                "eigenvector_centrality": eig_c.get(n, 0.0),
                "betweenness_centrality": betw.get(n, 0.0),
                "clustering": clust.get(n, 0.0),
                "modularity_class": node2comm.get(n, -1),
                "interlock_entropy_bits": entropy_bits.get(n, 0.0),
                "strength": strength.get(n, 0.0),
            })
        out.append(pd.DataFrame(rows))

    if not out:
        base_cols = ["Year","Id","Label","SystemId","degree","degree_centrality",
                     "eigenvector_centrality","betweenness_centrality",
                     "clustering","modularity_class","interlock_entropy_bits","strength"]
        return pd.DataFrame(columns=base_cols)

    metrics = pd.concat(out, ignore_index=True)

    # Join node attributes (Label, SystemId) -> single nodes_df with Year granularity
    node_attrs = nodes[["Id", "Label", "SystemId"]].drop_duplicates()
    nodes_updated = (
        metrics.merge(node_attrs, on="Id", how="left")
        .reindex(columns=["Year","Id","Label","SystemId",
                          "degree","degree_centrality","eigenvector_centrality",
                          "betweenness_centrality","clustering","modularity_class",
                          "interlock_entropy_bits","strength"])
        .sort_values(["Year","Id"], kind="mergesort")
        .reset_index(drop=True)
    )
    return nodes_updated




def attach_edge_entropy(edges, weight_col="WeightNorm", out_col="EdgeEntropyBits"):
    """
    Compute edge-level local entropy around endpoints and return UPDATED edges DF.
    For each (Year, u, v): H( concat( weights(u,·\{v}), weights(v,·\{u}) ) ).
    If no neighbors besides each other, entropy=0.
    """
    if edges.empty:
        edges[out_col] = np.nan
        return edges

    edges = edges.copy()
    edges["AffiliationId_1"] = edges["AffiliationId_1"].astype(object)
    edges["AffiliationId_2"] = edges["AffiliationId_2"].astype(object)

    out_chunks = []
    for year, gdf in edges.groupby("Year", sort=False):
        edge_attr = [weight_col] if (weight_col in gdf.columns) else None
        G = nx.from_pandas_edgelist(
            gdf,
            source="AffiliationId_1",
            target="AffiliationId_2",
            edge_attr=edge_attr,
            create_using=nx.Graph()
        )
        if G.number_of_edges() == 0:
            # should not hit
            tmp = gdf.copy()
            tmp[out_col] = 0.0
            out_chunks.append(tmp)
            continue

        ent_map = {}
        for u, v in G.edges():
            # neighbors around u and v (excluding the counterpart)
            wu = [float(G[u][nbr].get(weight_col, 1.0)) for nbr in G.neighbors(u) if nbr != v]
            wv = [float(G[v][nbr].get(weight_col, 1.0)) for nbr in G.neighbors(v) if nbr != u]
            ent_map[(min(u, v), max(u, v))] = _shannon_entropy_bits(wu + wv)

        tmp = gdf.copy()
        tmp[out_col] = [
            ent_map.get((min(u, v), max(u, v)), 0.0)
            for u, v in zip(tmp["AffiliationId_1"], tmp["AffiliationId_2"])
        ]
        out_chunks.append(tmp)

    return pd.concat(out_chunks, ignore_index=True)


<>:338: SyntaxWarning: invalid escape sequence '\{'
<>:338: SyntaxWarning: invalid escape sequence '\{'
C:\Users\tykun\AppData\Local\Temp\ipykernel_30244\3014831229.py:338: SyntaxWarning: invalid escape sequence '\{'
  """


In [6]:
# edges_df, nodes_df = build_interlock_graphs(board_df)
# edges_df = attach_weakness(edges_df, nodes_df)

# # Add simple aliases for convenience
# edges_df = edges_df.assign(
#     Source=edges_df["AffiliationId_1"],
#     Target=edges_df["AffiliationId_2"],
#     Weight=edges_df["WeightNorm"]
# )

In [7]:
# same as before
edges_df, nodes_df = build_interlock_graphs(board_df)
edges_df = attach_weakness(edges_df, nodes_df)

# your aliases
edges_df = edges_df.assign(
    Source=edges_df["AffiliationId_1"],
    Target=edges_df["AffiliationId_2"],
    Weight=edges_df["WeightNorm"]
)

# NEW: put all node metrics directly into nodes_df (now by Year)
nodes_df = attach_node_metrics(
    edges_df, nodes_df,
    weight_col="Weight",            # your alias, or "WeightNorm"
    use_weighted_betweenness=False  # set True to use distance = 1/weight
)

# OPTIONAL: add edge entropy to edges_df
edges_df = attach_edge_entropy(edges_df, weight_col="Weight", out_col="EdgeEntropyBits")


c:\Python312\Lib\site-packages\networkx\algorithms\centrality\eigenvector.py:352: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  _, eigenvector = sp.sparse.linalg.eigs(


In [8]:
edges_df.to_csv(os.path.join(data_path, 'interlocks', 'interlock_edges.csv'), index=False)
nodes_df.to_csv(os.path.join(data_path, 'interlocks', 'interlock_nodes.csv'), index=False)


In [9]:
edges_df, nodes_df = build_interlock_graphs(board_df)
edges_df = attach_weakness(edges_df, nodes_df)

# Add simple aliases for convenience
edges_df = edges_df.assign(
    Source=edges_df["AffiliationId_1"],
    Target=edges_df["AffiliationId_2"],
    Weight=edges_df["WeightNorm"]
)